# Common Setup

This notebook sets up the common configuration for the Databricks AI training course.

In [0]:
# Setup logging to lowest level. In dev, this can be changed
import warnings
import logging

# Hide warnings
warnings.filterwarnings("ignore")

# Limit Spark logs
spark.sparkContext.setLogLevel("ERROR")

# Limit other library logs
logging.getLogger().setLevel(logging.ERROR)

# Library-level logging setup
logging.getLogger("py4j").setLevel(logging.ERROR)

In [0]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress TensorFlow logs

import time
import re
import io

# Import PyPDF2 - should be installed by main notebook before this setup runs
# If not available, this will raise an error prompting the user to install it first
try:
    from PyPDF2 import PdfReader
except ImportError as e:
    raise ImportError(
        "PyPDF2 is not installed. Please ensure the main notebook has run the library installation cell "
        "(typically cell 4) which installs PyPDF2 before running this setup."
    ) from e

# Handle potential import error with accelerate
try:
    pass
except ImportError as e:
    print(f"Warning: {e}. MLU support might not be available in this environment.")

In [0]:
# Function used to randomly assign each user a VS Endpoint
def get_fixed_integer(string_input):
    # Calculate the sum of ASCII values of the characters in the input string
    ascii_sum = sum(ord(char) for char in string_input)
    
    # Map the sum to a fixed integer between 1 and 9
    fixed_integer = (ascii_sum % 5) + 1
    
    return fixed_integer

In [0]:
#parse PDF bytes using PyPDF2
def parse_bytes_pypdf(raw_doc_contents_bytes: bytes):
    try:
        pdf = io.BytesIO(raw_doc_contents_bytes)
        reader = PdfReader(pdf)
        parsed_content = [page_content.extract_text() for page_content in reader.pages]
        return "\n".join(parsed_content)
    except Exception as e:
        warnings.warn(f"Exception {e} has been thrown during parsing")
        return None 


def pprint(obj):
  import pprint
  pprint.pprint(obj, compact=True, indent=1, width=100)

def index_exists(vsc, endpoint_name, index_full_name):
  try:
      dict_vsindex = vsc.get_index(endpoint_name, index_full_name).describe()
      return dict_vsindex.get('status').get('ready', False)
  except Exception as e:
      if 'RESOURCE_DOES_NOT_EXIST' not in str(e) and 'NOT_FOUND' not in str(e):
          print(f'Unexpected error describing the index. This could be a permission issue.')
          raise e
  return False

def wait_for_vs_endpoint_to_be_ready(vsc, vs_endpoint_name):
  for i in range(180):
    endpoint = vsc.get_endpoint(vs_endpoint_name)
    status = endpoint.get("endpoint_status", endpoint.get("status"))["state"].upper()
    if "ONLINE" in status:
      return endpoint
    elif "PROVISIONING" in status or i <6:
      if i % 20 == 0: 
        print(f"Waiting for endpoint to be ready, this can take a few min... {endpoint}")
      time.sleep(10)
    else:
      raise Exception(f'''Error with the endpoint {vs_endpoint_name}. - this shouldn't happen: {endpoint}.\n Please delete it and re-run the previous cell: vsc.delete_endpoint("{vs_endpoint_name}")''')
  raise Exception(f"Timeout, your endpoint isn't ready yet: {vsc.get_endpoint(vs_endpoint_name)}")

def wait_for_index_to_be_ready(vsc, vs_endpoint_name, index_name):
  for i in range(180):
    idx = vsc.get_index(vs_endpoint_name, index_name).describe()
    index_status = idx.get('status', idx.get('index_status', {}))
    status = index_status.get('detailed_state', index_status.get('status', 'UNKNOWN')).upper()
    url = index_status.get('index_url', index_status.get('url', 'UNKNOWN'))
    if "ONLINE" in status:
      return
    if "UNKNOWN" in status:
      print(f"Can't get the status - will assume index is ready {idx} - url: {url}")
      return
    elif "PROVISIONING" in status:
      if i % 40 == 0: print(f"Waiting for index to be ready, this can take a few min... {index_status} - pipeline url:{url}")
      time.sleep(10)
    else:
        raise Exception(f'''Error with the index - this shouldn't happen. DLT pipeline might have been killed.\n Please delete it and re-run the previous cell: vsc.delete_index("{index_name}, {vs_endpoint_name}") \nIndex details: {idx}''')
  raise Exception(f"Timeout, your index isn't ready yet: {vsc.get_index(index_name, vs_endpoint_name)}")